In [19]:
import os
from openpyxl.workbook import Workbook
from sklearn import clone
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imbens.ensemble import BalancedRandomForestClassifier
from imbens.ensemble import EasyEnsembleClassifier
from imbens.ensemble import KmeansSMOTEBoostClassifier
from instance_selection.operator.metrics import calculate_gmean_mauc
from instance_selection.parameter.parameter import *  # 导入参数的设定
import scipy.io as sio  # 从.mat文件中读取数据集


def data_process(dataset=None):
    mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + dataset.DATASETNAME)  # 加载、划分数据集
    x = mat_data['X']
    y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y,
                                                        random_state=RANDOM_SEED)  # 划分数据集
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    model = MLPClassifier(hidden_layer_sizes=(dataset.HIDDEN_SIZE,), max_iter=dataset.MAX_ITER,
                          random_state=RANDOM_SEED, learning_rate_init=dataset.LEARNING_RATE)
    return x_train, x_test, y_train, y_test, clone(model)


def save_to_excel(data, save_path, filename='avg_results'):
    """
    将列表数据逐行写入Excel文件
    参数:
        data: 二维列表，每个子列表代表一行数据
        filename: 输出的Excel文件名(默认为output.xlsx)
    """
    # 创建一个新的工作簿
    wb = Workbook()
    # 获取活动的工作表
    ws = wb.active
    # 逐行写入数据
    for row in data:
        avg = row[1].tolist()
        avg.insert(0, row[0])
        std = row[2].tolist()
        std.insert(0, row[0])
        ws.append(avg)
        ws.append(std)
    # 创建 Excel 文件完整路径
    file_path = os.path.join(save_path, filename + ".xlsx")
    # 保存Excel文件
    wb.save(file_path)
    print(f"数据已成功写入到 {file_path}")

In [20]:

from imbens.ensemble import BalanceCascadeClassifier, CompatibleBaggingClassifier, RUSBoostClassifier, \
    SMOTEBaggingClassifier, SMOTEBoostClassifier
from utils.excel_utils import save_to_excel_2
import numpy as np

DATASETS = [Balance_Scale, Dermatology, Ecoli, Car, Pen_Digits, WallRobot, German, Wine, Nursery, Penbased, USPS,
            Satellite, Page_Blocks, Shuttle, Contraceptive, Automobile, Ovarian]  # 数据集名称（包含对应的参数配置）

if __name__ == "__main__":
    save_path = 'C:/Users/zsc/Desktop/SMOTEBoost+MLP/'
    columns = ['Gmean', 'MAUC']
    datasets_ensembles_results = [[] for _ in range(len(DATASETS))]
    print("*****************算法开始执行：******************")
    for j, dataset in enumerate(DATASETS):
        x_train, x_test, y_train, y_test, model = data_process(dataset=dataset)
        num_run = 40  # 运行次数
        ensembles_results = [[] for _ in range(num_run)]
        for i in range(num_run):
            # clf = EasyEnsembleClassifier(estimator=clone(model), random_state=42 + np.random.randint(0, 1000),
            #                              n_estimators=30) # EasyEnsembleClassifier
            # clf = BalancedRandomForestClassifier(n_estimators=30, random_state=42 + np.random.randint(0, 1000))
            # clf = KmeansSMOTEBoostClassifier(random_state=42 + np.random.randint(0, 1000),
            #                                 n_estimators=30)
            # clf = BalanceCascadeClassifier(estimator=clone(model), random_state=42 + np.random.randint(0, 1000),
            #                               n_estimators=30) 
            clf = SMOTEBoostClassifier(random_state=42 + np.random.randint(0, 1000),
                                          n_estimators=30) 
            clf.fit(x_train, y_train)
            y_pred_prob = clf.predict_proba(x_test)  # 默认预测结果是软标签
            gmean, mauc, recall_per_class = calculate_gmean_mauc(y_pred_prob, y_test)

            ensembles_results[i] = [gmean, mauc]
            print(f"第{i + 1}次执行：Gmean：{gmean}，mAUC：{mauc}")
        save_to_excel_2(save_path + dataset.DATASETNAME.split('.')[0] + '/', dataset.DATASETNAME.split('.')[0], columns,
                        ensembles_results)
        ensembles_result_mean = np.mean(ensembles_results, axis=0)
        # 计算ensembles_resultsz中每一列的标准差
        ensembles_result_std = np.std(ensembles_results, axis=0)
        print(f'集成分类结果（平均值）：{ensembles_result_mean}')
        print(f'集成分类结果（标准差）：{ensembles_result_std}')
        datasets_ensembles_results[j] = [dataset.DATASETNAME.split('.')[0], ensembles_result_mean, ensembles_result_std]
    print("*****************算法执行结束！******************")
    # 写入到Excel     
    save_to_excel(datasets_ensembles_results, save_path)       

*****************算法开始执行：******************
第1次执行：Gmean：0.689435，mAUC：0.778338
第2次执行：Gmean：0.656479，mAUC：0.782078
第3次执行：Gmean：0.697476，mAUC：0.782154
第4次执行：Gmean：0.680458，mAUC：0.775617
第5次执行：Gmean：0.682249，mAUC：0.790175
第6次执行：Gmean：0.630577，mAUC：0.793158
第7次执行：Gmean：0.679584，mAUC：0.819671
第8次执行：Gmean：0.668643，mAUC：0.785139
第9次执行：Gmean：0.750543，mAUC：0.79719
第10次执行：Gmean：0.765786，mAUC：0.820304
第11次执行：Gmean：0.688321，mAUC：0.784582
第12次执行：Gmean：0.670743，mAUC：0.779209
第13次执行：Gmean：0.627424，mAUC：0.803502
第14次执行：Gmean：0.69404，mAUC：0.775486
第15次执行：Gmean：0.707704，mAUC：0.800221
第16次执行：Gmean：0.679018，mAUC：0.78775
第17次执行：Gmean：0.662975，mAUC：0.795431
第18次执行：Gmean：0.633699，mAUC：0.782505
第19次执行：Gmean：0.604896，mAUC：0.766327
第20次执行：Gmean：0.6703，mAUC：0.775858
第21次执行：Gmean：0.670366，mAUC：0.786084
第22次执行：Gmean：0.647479，mAUC：0.779722
第23次执行：Gmean：0.645931，mAUC：0.783059
第24次执行：Gmean：0.688321，mAUC：0.784582
第25次执行：Gmean：0.638864，mAUC：0.778514
第26次执行：Gmean：0.642626，mAUC：0.789106
第27次执行：Gmean：0.655996，mAUC：0.78431
